In [ ]:
import torch
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_data=MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
print(len(train_data),len(test_data))

60000 10000


In [ ]:
image,label=train_data[0]
image.shape

torch.Size([1, 28, 28])

In [ ]:
train_ds,val_ds=random_split(train_data,[50000,10000])
len(train_ds),len(val_ds)

(50000, 10000)

In [ ]:
batch_size=128
train_loader=DataLoader(train_ds,batch_size,shuffle=True)
val_loader=DataLoader(val_ds,batch_size)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
def fit(epochs,lr,model,train_loader,val_loader):
  optimizer=torch.optim.SGD(model.parameters(),lr)
  history=[]
  for epoch in range(epochs):
    for batch in train_loader:
      loss=model.training_step(batch)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    result=evaluate(model,val_loader)
    model.epoch_end(epoch,result)
    history.append(result)
  return history

In [ ]:
def evaluate(model,val_loader):
  outputs=[model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

In [ ]:
class MyNN(nn.Module):
  def __init__ (self):
    super().__init__()
    self.linear=nn.Linear(28*28,10)
    self.relu=nn.ReLU()
  def forward(self,x):
    x=x.reshape(-1,784)
    out=self.linear(x)
    return out
  def training_step(self,batch):
    images,labels=batch
    out=self(images)
    loss=F.cross_entropy(out,labels)
    return loss
  def validation_step(self,batch):
     images,labels=batch
     out=self(images)
     loss=F.cross_entropy(out,labels)
     acc=accuracy(out,labels)
     return {'val_loss':loss,"val_acc":acc}
  def validation_epoch_end(self,outputs):
    batch_losses=[x['val_loss'] for x in outputs]
    epoch_loss=torch.stack(batch_losses).mean()
    batch_accs=[x['val_acc'] for x in outputs]
    epoch_acc=torch.stack(batch_accs).mean()
    return {'val_loss':epoch_loss.item(),'val_acc':epoch_acc.item()}
  def epoch_end(self,epoch,result):
    print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
model=MyNN()

In [ ]:
result1=evaluate(model,val_loader)
result1

{'val_loss': 2.3093972206115723, 'val_acc': 0.14636075496673584}

In [ ]:
h1=fit(5,0.001,model,train_loader,val_loader)

Epoch [0], val_loss: 1.9370, val_acc: 0.6351
Epoch [1], val_loss: 1.6712, val_acc: 0.7199
Epoch [2], val_loss: 1.4741, val_acc: 0.7544
Epoch [3], val_loss: 1.3256, val_acc: 0.7760
Epoch [4], val_loss: 1.2114, val_acc: 0.7888


In [ ]:
h2=fit(15,0.001,model,train_loader,val_loader)

Epoch [0], val_loss: 1.1218, val_acc: 0.7999
Epoch [1], val_loss: 1.0498, val_acc: 0.8070
Epoch [2], val_loss: 0.9910, val_acc: 0.8119
Epoch [3], val_loss: 0.9419, val_acc: 0.8165
Epoch [4], val_loss: 0.9004, val_acc: 0.8203
Epoch [5], val_loss: 0.8648, val_acc: 0.8231
Epoch [6], val_loss: 0.8339, val_acc: 0.8280
Epoch [7], val_loss: 0.8069, val_acc: 0.8316
Epoch [8], val_loss: 0.7830, val_acc: 0.8338
Epoch [9], val_loss: 0.7617, val_acc: 0.8371
Epoch [10], val_loss: 0.7425, val_acc: 0.8390
Epoch [11], val_loss: 0.7254, val_acc: 0.8406
Epoch [12], val_loss: 0.7097, val_acc: 0.8429
Epoch [13], val_loss: 0.6954, val_acc: 0.8452
Epoch [14], val_loss: 0.6822, val_acc: 0.8460
